In [106]:
from pathlib import Path
import getpass

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
import rasterio.plot
from rasterio import features
import datetime as dt

from sentinelhub import (
    SHConfig,
    CRS,
    BBox,
    DataCollection,
    DownloadRequest,
    MimeType,
    MosaickingOrder,
    SentinelHubDownloadClient,
    SentinelHubStatisticalDownloadClient,
    SentinelHubRequest,
    bbox_to_dimensions,
    SentinelHubStatistical,
    Geometry,
    parse_time,
)

In [107]:
# Commented out to avoid overwriting existing config
# Comment in to create new config:

# config = SHConfig()
# config.sh_client_id = getpass.getpass("Enter your SentinelHub client id")
# config.sh_client_secret = getpass.getpass("Enter your SentinelHub client secret")
# config.sh_token_url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
# config.sh_base_url = "https://sh.dataspace.copernicus.eu"
# config.save("cdse")
config = SHConfig("cdse")

In [108]:
# We also need to define the evalscript as a Python variable
evalscript_raw = """
//VERSION=3
function setup() {
   return {
    input: ["NO2", "dataMask"],
    output: 
      {
        id: "default",
        bands: 1,
        sampleType: "FLOAT32"
      },
    mosaicking: "ORBIT"
  };
}
function evaluatePixel(sample) {
  if (sample.dataMask == 1)  {
    return [sample.NO2];
  } else {
    return [NaN];
  }
}
"""

In [119]:
# define area of interest
cologne_coords_wgs84 = [6.8, 50.8, 7.2, 51.1]

ruhrgebiet_coords_wgs84 = [
  6.380946,
  51.315164,
  7.93203,
  51.738085
]

germany_coords_wgs84 = [5.866315, 47.270111, 15.041896, 55.099161]
aoi_bbox = BBox(bbox=germany_coords_wgs84, crs=CRS.WGS84).transform(CRS(3857))

In [120]:
# define time interval
time_range = ("2020-01-01", "2020-01-31")

start, end = pd.to_datetime(time_range[0]), pd.to_datetime(time_range[1])

daily_intervals = [
    (
        day.strftime("%Y-%m-%dT00:00:00Z"),
        day.strftime("%Y-%m-%dT23:59:59Z")
    )
    for day in pd.date_range(start, end, freq="D")
]

In [122]:
# This is defining the data we will use.
# You can list all available data collections with `DataCollection.get_available_collections()`.
data_5p = DataCollection.SENTINEL5P.define_from("5p", service_url=config.sh_base_url)

request_raw = SentinelHubRequest(
    evalscript=evalscript_raw,
    input_data=[
        SentinelHubRequest.input_data(
            data_collection=data_5p,
            time_interval=(daily_intervals[0][0], daily_intervals[0][1]),
        )
    ],
    responses=[SentinelHubRequest.output_response("default", MimeType.TIFF)],
    bbox=aoi_bbox,
    # Resolution is defined in units of the bbox crs! Be careful with WGS84 since this will be in degrees!
    # Since we have defined our bounding box in Web mercator the resolution is in meters.
    resolution=(5500, 3500),
    config=config,
    data_folder="./data",  # We save the data in a specified folder
)

In [123]:
raw_data = request_raw.get_data(save_data=True, redownload=True)

For contextual mapping and spatial aggregation, this notebook uses publicly available administrative boundary shapefiles from the Natural Earth dataset (Admin 0 – Countries, 1:50m scale). These files contain the polygon geometries of countries and are used only for visualization and zonal statistics. They are not part of the Sentinel-5P satellite data.

[Download Link](https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/50m/cultural/ne_50m_admin_0_countries.zip)

In [124]:
raw_data[0].shape

(399, 186)

In [126]:
raw_data[0].sum()

np.float32(nan)